In [2]:
from bs4 import BeautifulSoup 
import requests
import numpy as np
import pandas as pd
import re

In [68]:

url = "https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory"

text = requests.get(url).text



In [14]:
countries = "Austria, Belgium, Bulgaria, Croatia, Cyprus, Czechia, Czech Republic, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Spain, Sweden"
eu_countries = countries.split(", ")

In [67]:
def parse_language(repl):
    return re.sub("\(([^\)]+)\)", "", repl.strip().split("[")[0].replace("\n", ", ")).strip()

In [70]:
soup = BeautifulSoup(text)
raw_data = soup.find_all("table")[0].find_all("tbody")[0].find_all("tr")[1:]

languages = pd.DataFrame()
for row in raw_data:
    data = row.find_all("td")
    country = parse_language(data[0].text)
    official_lang = parse_language(data[1].text).split(", ")
    minority_lang = parse_language(data[3].text).split("," )
    widely_spoken_lang = ""

    if (len(data) >= 6):
        widely_spoken_lang = parse_language(data[5].text).split(", ")
    
    

    if country in eu_countries:

        row = {"country" : country, "official_lang" : official_lang, "minority_lang" : minority_lang, "widely_spoken_lang": widely_spoken_lang}
        row_series = pd.Series(row)
        languages = pd.concat([languages, row_series.to_frame().T])

languages


,country,official_lang,minority_lang,widely_spoken_lang
0,Austria,[German],"[Slovene, Czech, Hungarian, Slovak, Romani...",[English]
0,Belgium,"[Dutch, French, German]",[],[English]
0,Bulgaria,[Bulgarian],[],[]
0,Croatia,[Croatian],[],[English]
0,Cyprus,[Greek],[],[English]
0,Czech Republic,[Czech],[Belarusian],[]
0,Denmark,[Danish],[German],[English]
0,Estonia,[Estonian],[],[Russian]
0,Finland,"[Finnish, Swedish]",[Sami],[English]
0,France,[French],"[Corsican language, Basque language, Breton ...",[Occitan language]


In [79]:
url = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density"

html = requests.get(url).text

pop_dens_soup = BeautifulSoup(html)

rows = pop_dens_soup.find_all("tbody")[0]

rows = rows.find_all("tr")[1:]

countries = []
pop_density = []


for row in rows:
    data = row.find_all("td")
    country = data[0].text.strip()
    pop_dens_km = float(data[1].text.strip().replace(",", ""))

    #special case, need specifically metropolitan france
    if country ==  "France (metropolitan)":
        country = "France"
    
    if country in eu_countries:
        countries.append(country)
        pop_density.append(pop_dens_km)

pop_density_df = pd.DataFrame({"country" : countries, "pop_density" : pop_density})
pop_density_df

    



,country,pop_density
0,Malta,1700.0
1,Netherlands,424.0
2,Belgium,383.0
3,Luxembourg,253.0
4,Germany,233.0
5,Italy,195.0
6,Denmark,138.0
7,Cyprus,136.0
8,Czech Republic,133.0
9,Poland,120.0


Units are per km^2

In [6]:
url  = "https://worldpopulationreview.com/country-rankings/best-healthcare-in-the-world"

html = requests.get(url).text

healthcare_soup = BeautifulSoup(html)


<table class="tp-table-body is-narrow w-full min-w-full table-auto border-separate border-spacing-0 border bg-white"><thead class="relative z-40 bg-sky-100 px-4 py-3 font-bold"><tr><th class="datatable-th align-bottom sticky top-0 border-b border-gray-400 bg-sky-50 bg-clip-padding px-2 py-1 pin left-0 z-50 border-l-0 text-left"><div class="flex flex-col justify-end items-stretch"><div class="flex items-center cursor-pointer select-none">Country</div><div><div class="flex flex-col"><input class="w-36 border shadow rounded-sm px-1 py-0.5 text-sm" placeholder="Search... (195)" type="text" value=""/><div class="h-1"></div></div></div></div></th><th class="datatable-th align-bottom sticky top-0 border-b border-gray-400 bg-sky-50 bg-clip-padding px-2 py-1 z-40"><div class="flex flex-col justify-end items-stretch"><div class="flex items-center cursor-pointer select-none">Legatum Prosperity Index Health Score<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="18" stroke=

In [74]:
countries = []
scores = []

for tr in healthcare_soup.find_all("table")[1].find_all("tr")[1:]:
    country = tr.find("th").text
    score = tr.find_all("td")[1].text
    if country in eu_countries:
        countries.append(country)
        scores.append(score)

healthcare_index_df = pd.DataFrame({"country" : countries, "healthcare_index" : scores})
healthcare_index_df

,country,healthcare_index
0,Sweden,56.29
1,Netherlands,54.63
2,Luxembourg,56.31
3,Germany,55.98
4,Finland,60.53
5,Denmark,55.46
6,Italy,72.15
7,Belgium,53.99
8,Malta,53.59
9,France,55.31


In [37]:
url = "https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2024"

html = requests.get(url).text

col_soup = BeautifulSoup(html)

In [75]:
countries = []
scores = []

for tr in col_soup.find_all("table")[1].find_all("tr")[1:]:
    data = tr.find_all("td")
    
    
    country = data[1].text
    col = data[2].text
    if country in eu_countries:
        countries.append(country)
        scores.append(col)

COL_df = pd.DataFrame({"country" : countries, "COL_index" : scores})
COL_df

,country,COL_index
0,Denmark,75.2
1,Austria,66.8
2,Ireland,66.5
3,France,66.0
4,Finland,65.5
5,Luxembourg,65.3
6,Netherlands,64.3
7,Belgium,63.0
8,Germany,62.7
9,Sweden,62.2


Units: arbitrary, its an index

Getting Temperature data

In [39]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature"

html = requests.get(url).text

temp_soup = BeautifulSoup(html)

In [77]:
countries = []
temps = []
for tr in temp_soup.find_all("tbody")[0].find_all("tr")[1:]:
    data = tr.find_all("td")
    country = data[1].text.strip()
    temp = re.sub(r'[^0-9.]', "", data[2].text.split(" ")[0])
    if country in eu_countries:
        countries.append(country)
        temps.append(temp)

temperature_df = pd.DataFrame({"country" : countries, "temperatures" : temps})
temperature_df


,country,temperatures
0,Malta,19.58
1,Cyprus,18.95
2,Portugal,15.85
3,Greece,14.26
4,Spain,14.07
5,Italy,13.22
6,Croatia,11.96
7,France,11.65
8,Hungary,11.50
9,Bulgaria,11.35


In degrees C

In [84]:
dfs = [temperature_df, COL_df, healthcare_index_df, pop_density_df, languages]

merged_df = pd.merge(temperature_df, COL_df, left_on="country", right_on="country")
for df in dfs[2:]:
    merged_df = pd.merge(merged_df, df, left_on="country", right_on="country")

merged_df.to_csv("./Data/wiki_data.csv", index=False)
merged_df



,country,temperatures,COL_index,healthcare_index,pop_density,official_lang,minority_lang,widely_spoken_lang
0,Malta,19.58,55.1,53.59,1700.0,"[Maltese, English]",[],[]
1,Cyprus,18.95,57.1,53.74,136.0,[Greek],[],[English]
2,Portugal,15.85,46.6,54.95,111.0,[Portuguese],[],[English]
3,Greece,14.26,54.2,52.5,78.0,[Greek],[],[]
4,Spain,14.07,48.4,53.55,97.0,[Spanish],"[Astur-Leonese , Aragonese , Fala , French,...",[]
5,Italy,13.22,58.9,72.15,195.0,[Italian],"[Ladin, Slovene]",[Other Italo-Dalmatian languages]
6,Croatia,11.96,46.9,52.4,71.0,[Croatian],[],[English]
7,France,11.65,66.0,55.31,117.0,[French],"[Corsican language, Basque language, Breton ...",[Occitan language]
8,Hungary,11.50,43.4,49.97,109.0,[Hungarian],[Croatian],[]
9,Bulgaria,11.35,38.7,47.02,61.0,[Bulgarian],[],[]
